In [1]:
import argparse

import os.path as osp
import time

import torch
import torch.nn.functional as F

import torch_geometric.transforms as T
from torch_geometric.datasets import Planetoid
from torch_geometric.logging import init_wandb, log
from torch_geometric.nn import GCNConv
import pandas as pd
import sys
sys.argv = ['try_using GCN,ipynb']

In [2]:
df_credit_card = pd.read_csv('./creditcard.csv')
df_credit_card

,Time,V1,V2,V3,V4,V5,V6,V7,V8,V9,...,V21,V22,V23,V24,V25,V26,V27,V28,Amount,Class
0,0.0,-1.359807,-0.072781,2.536347,1.378155,-0.338321,0.462388,0.239599,0.098698,0.363787,...,-0.018307,0.277838,-0.110474,0.066928,0.128539,-0.189115,0.133558,-0.021053,149.62,0
1,0.0,1.191857,0.266151,0.166480,0.448154,0.060018,-0.082361,-0.078803,0.085102,-0.255425,...,-0.225775,-0.638672,0.101288,-0.339846,0.167170,0.125895,-0.008983,0.014724,2.69,0
2,1.0,-1.358354,-1.340163,1.773209,0.379780,-0.503198,1.800499,0.791461,0.247676,-1.514654,...,0.247998,0.771679,0.909412,-0.689281,-0.327642,-0.139097,-0.055353,-0.059752,378.66,0
3,1.0,-0.966272,-0.185226,1.792993,-0.863291,-0.010309,1.247203,0.237609,0.377436,-1.387024,...,-0.108300,0.005274,-0.190321,-1.175575,0.647376,-0.221929,0.062723,0.061458,123.50,0
4,2.0,-1.158233,0.877737,1.548718,0.403034,-0.407193,0.095921,0.592941,-0.270533,0.817739,...,-0.009431,0.798278,-0.137458,0.141267,-0.206010,0.502292,0.219422,0.215153,69.99,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
284802,172786.0,-11.881118,10.071785,-9.834783,-2.066656,-5.364473,-2.606837,-4.918215,7.305334,1.914428,...,0.213454,0.111864,1.014480,-0.509348,1.436807,0.250034,0.943651,0.823731,0.77,0
284803,172787.0,-0.732789,-0.055080,2.035030,-0.738589,0.868229,1.058415,0.024330,0.294869,0.584800,...,0.214205,0.924384,0.012463,-1.016226,-0.606624,-0.395255,0.068472,-0.053527,24.79,0
284804,172788.0,1.919565,-0.301254,-3.249640,-0.557828,2.630515,3.031260,-0.296827,0.708417,0.432454,...,0.232045,0.578229,-0.037501,0.640134,0.265745,-0.087371,0.004455,-0.026561,67.88,0
284805,172788.0,-0.240440,0.530483,0.702510,0.689799,-0.377961,0.623708,-0.686180,0.679145,0.392087,...,0.265245,0.800049,-0.163298,0.123205,-0.569159,0.546668,0.108821,0.104533,10.00,0


In [3]:
from torch_geometric.data import Data
x = torch.tensor(df_credit_card.iloc[:, :30].values, dtype=torch.float)
y= torch.tensor(df_credit_card['Class'].values, dtype=torch.long)

In [4]:
num_nodes = 30
edges = []
for i in range(num_nodes):
    for j in range(i + 1, num_nodes):
        edges.append([i, j])

edge_index = torch.tensor(edges, dtype=torch.long).t().contiguous()
edge_index

tensor([[ 0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
          0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  1,  1,  1,  1,  1,  1,  1,
          1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,
          1,  1,  1,  2,  2,  2,  2,  2,  2,  2,  2,  2,  2,  2,  2,  2,  2,  2,
          2,  2,  2,  2,  2,  2,  2,  2,  2,  2,  2,  2,  3,  3,  3,  3,  3,  3,
          3,  3,  3,  3,  3,  3,  3,  3,  3,  3,  3,  3,  3,  3,  3,  3,  3,  3,
          3,  3,  4,  4,  4,  4,  4,  4,  4,  4,  4,  4,  4,  4,  4,  4,  4,  4,
          4,  4,  4,  4,  4,  4,  4,  4,  4,  5,  5,  5,  5,  5,  5,  5,  5,  5,
          5,  5,  5,  5,  5,  5,  5,  5,  5,  5,  5,  5,  5,  5,  5,  6,  6,  6,
          6,  6,  6,  6,  6,  6,  6,  6,  6,  6,  6,  6,  6,  6,  6,  6,  6,  6,
          6,  6,  7,  7,  7,  7,  7,  7,  7,  7,  7,  7,  7,  7,  7,  7,  7,  7,
          7,  7,  7,  7,  7,  7,  8,  8,  8,  8,  8,  8,  8,  8,  8,  8,  8,  8,
          8,  8,  8,  8,  8,

In [5]:
import numpy as np
total_samples = 284807

# 划分比例
train_ratio = 0.7
val_ratio = 0.2
test_ratio = 0.1

# 划分数量
num_train = int(train_ratio * total_samples)
num_val = int(val_ratio * total_samples)
num_test = int(test_ratio * total_samples)

# 创建掩码
train_mask = np.zeros(total_samples)
val_mask = np.zeros(total_samples)
test_mask = np.zeros(total_samples)

# 将相应数量的节点置为1
train_mask[:num_train] = 1
val_mask[num_train:num_train + num_val] = 1
test_mask[num_train + num_val:num_train + num_val + num_test] = 1

# 打印掩码中 1 的数量，确保划分正确
print("Train nodes:", np.sum(train_mask))
print("Validation nodes:", np.sum(val_mask))
print("Test nodes:", np.sum(test_mask))

Train nodes: 199364.0
Validation nodes: 56961.0
Test nodes: 28480.0


In [6]:
data = Data(x=x, y=y, edge_index=edge_index, train_mask=train_mask, val_mask=val_mask, test_mask=test_mask)
data

Data(x=[284807, 30], edge_index=[2, 435], y=[284807], train_mask=[284807], val_mask=[284807], test_mask=[284807])

In [7]:
parser = argparse.ArgumentParser()
parser.add_argument('--dataset', type=str, default='Cora')
parser.add_argument('--hidden_channels', type=int, default=16)
parser.add_argument('--lr', type=float, default=0.01)
parser.add_argument('--epochs', type=int, default=200)
parser.add_argument('--use_gdc',  action='store_true', help='Use GDC')
parser.add_argument('--wandb', action='store_true', help='Track experiment')
args = parser.parse_args()

In [8]:
if torch.cuda.is_available():
    device = torch.device('cuda')
elif hasattr(torch.backends, 'mps') and torch.backends.mps.is_available():
    device = torch.device('mps')
else:
    device = torch.device('cpu')

In [9]:
init_wandb(
    name='credit_fraud',
    lr = args.lr,
    epochs=args.epochs,
    hidden_channels=args.hidden_channels,
    device=device,
)

In [10]:
if args.use_gdc:
    transform = T.GDC(
        self_loop_weight=1,
        normalization_in='sym',
        normalization_out='col',
        diffusion_kwargs=dict(method='ppr', alpha=0.05),
        sparsification_kwargs=dict(method='topk', k=128, dim=0),
        exact=True,
    )
    data = transform(data)

In [11]:
device

device(type='cuda')

In [14]:
class GCN(torch.nn.Module):
    def __init__(self, in_channels, hidden_channels, out_channels):
        super().__init__()
        self.conv1 = GCNConv(in_channels, hidden_channels,
                             normalize=not args.use_gdc)
        self.conv2 = GCNConv(hidden_channels, out_channels,
                             normalize=not args.use_gdc)

    def forward(self, x, edge_index, edge_weight=None):
        x = F.dropout(x, p=0.5, training=self.training)
        x = self.conv1(x, edge_index, edge_weight).relu()
        x = F.dropout(x, p=0.5, training=self.training)
        x = self.conv2(x, edge_index, edge_weight)
        return x


model = GCN(
    in_channels=30,
    hidden_channels=args.hidden_channels,
    out_channels=1,
)

optimizer = torch.optim.Adam([
    dict(params=model.conv1.parameters(), weight_decay=5e-4),
    dict(params=model.conv2.parameters(), weight_decay=0)
], lr=args.lr)  # Only perform weight-decay on first convolution.

def train():
    model.train()
    optimizer.zero_grad()
    out = model(data.x, data.edge_index, data.edge_attr)
    loss = F.cross_entropy(out[data.train_mask], data.y[data.train_mask])
    loss.backward()
    optimizer.step()
    return float(loss)


@torch.no_grad()
def test():
    model.eval()
    pred = model(data.x, data.edge_index, data.edge_attr).argmax(dim=-1)

    accs = []
    for mask in [data.train_mask, data.val_mask, data.test_mask]:
        accs.append(int((pred[mask] == data.y[mask]).sum()) / int(mask.sum()))
    return accs

best_val_acc = test_acc = 0
times = []
for epoch in range(1, args.epochs + 1):
    start = time.time()
    loss = train()
    train_acc, val_acc, tmp_test_acc = test()
    if val_acc > best_val_acc:
        best_val_acc = val_acc
        test_acc = tmp_test_acc
    log(Epoch=epoch, Loss=loss, Train=train_acc, Val=val_acc, Test=test_acc)
    times.append(time.time() - start)
print(f'Median time per epoch: {torch.tensor(times).median():.4f}s')

Epoch: 001, Loss: 1.7296, Train: 0.0000, Val: 0.0000, Test: 000
Epoch: 002, Loss: 0.6423, Train: 0.0000, Val: 0.0000, Test: 000
Epoch: 003, Loss: 0.8304, Train: 0.0000, Val: 0.0000, Test: 000
Epoch: 004, Loss: 0.4714, Train: 0.0000, Val: 0.0000, Test: 000
Epoch: 005, Loss: 31.2860, Train: 0.0000, Val: 0.0000, Test: 000
Epoch: 006, Loss: 0.0000, Train: 1.4286, Val: 5.0000, Test: 10.0002
Epoch: 007, Loss: 13.5889, Train: 1.4286, Val: 5.0000, Test: 10.0002
Epoch: 008, Loss: 0.0000, Train: 1.4286, Val: 5.0000, Test: 10.0002
Epoch: 009, Loss: 5.3508, Train: 1.4286, Val: 5.0000, Test: 10.0002
Epoch: 010, Loss: 0.1758, Train: 1.4286, Val: 5.0000, Test: 10.0002
Epoch: 011, Loss: 0.0000, Train: 1.4286, Val: 5.0000, Test: 10.0002
Epoch: 012, Loss: 0.0036, Train: 1.4286, Val: 5.0000, Test: 10.0002
Epoch: 013, Loss: 0.0000, Train: 1.4286, Val: 5.0000, Test: 10.0002
Epoch: 014, Loss: 0.2830, Train: 1.4286, Val: 5.0000, Test: 10.0002
Epoch: 015, Loss: 0.4394, Train: 1.4286, Val: 5.0000, Test: 10.000